In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
x_train = pd.read_csv('/kaggle/input/titanic/train.csv')
x_test = pd.read_csv('/kaggle/input/titanic/test.csv')
y_train = np.array(x_train['Survived'].copy())
id_test = np.array(x_test['PassengerId'].copy())

In [7]:
train_data.head()

In [8]:
x_test.head()

In [9]:
#count the number of elements in the dataset
print(train_data.size)

#show number of dimensions of the df
print(train_data.ndim)

#show dataset dimensions 
print(train_data.shape)

#show columns names
print(train_data.columns)

#count non null values and show column type
print(train_data.info(verbose=True))

In [10]:
#count the number of elements in the dataset
print(x_test.size)

#show number of dimensions of the df
print(x_test.ndim)

#show dataset dimensions 
print(x_test.shape)

#show columns names
print(x_test.columns)

#count non null values and show column type
print(x_test.info(verbose=True))

In [11]:
#show duplicated rows
train_data[train_data.duplicated()]

In [12]:
# 687 Cabin value of 891 are missing ==> drop it
x_train.drop(columns={'Cabin','PassengerId','Name','Survived','Ticket'}, inplace=True)
x_train.head()

In [13]:
x_test.drop(columns={'Cabin','PassengerId','Name','Ticket'}, inplace=True)
x_test.head()

In [14]:
x_train.describe() # statistical description


In [15]:
x_test.describe() # statistical description


In [16]:
train_data.Survived.value_counts().plot.bar()

In [17]:
x_train['Embarked'].value_counts(dropna=False) #count unique values in a column


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

num_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])


cat_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy="most_frequent")),
    ('encode', OneHotEncoder())
])


num_labels = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
cat_labels = ["Sex", "Embarked"]

full_pipeline = ColumnTransformer([
    ("num_trans", num_pipeline, num_labels),
    ("cat_trans", cat_pipeline, cat_labels)
])

In [19]:
x_train_prepared = full_pipeline.fit_transform(x_train)

In [20]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(C= 0.1, penalty= 'l2')
log_reg.fit(x_train_prepared, y_train)


In [21]:
log_reg.score(x_train_prepared, y_train)


In [22]:
x_test_prepared = full_pipeline.transform(x_test)
predicts = log_reg.predict(x_test_prepared)
predictss = pd.DataFrame(predicts)
predictss.rename( columns={0 :'Survived'}, inplace=True )
x_tests = pd.read_csv('/kaggle/input/titanic/test.csv')

ffff = pd.concat([x_tests['PassengerId'],predictss],axis=1)
final = ffff[['PassengerId','Survived']]

In [23]:
from IPython.display import HTML

final.to_csv('submission.csv', index=False)

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')

In [24]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier()
forest_clf.fit(x_train_prepared, y_train)

In [25]:
forest_clf.score(x_train_prepared, y_train)

In [26]:
# Searching for better parameters for random forest
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[150, 200, 250], 'max_features':[5, 10], 'max_depth':[5, 10, 20], 
     'min_samples_leaf':[5, 10, 20], 'min_samples_split':[5, 10, 20]}
]
 
forest_clf = RandomForestClassifier()
grid_search = GridSearchCV(forest_clf, param_grid, cv=3, scoring='accuracy', return_train_score=True)
grid_search.fit(x_train_prepared, y_train)
best_search_rf = grid_search.best_estimator_
print(best_search_rf)
grid_search.best_params_

In [28]:
best_search_rf.fit(x_train_prepared, y_train)

In [29]:
best_search_rf.score(x_train_prepared, y_train)

In [30]:
predicts = best_search_rf.predict(x_test_prepared)
predictss = pd.DataFrame(predicts)
predictss.rename( columns={0 :'Survived'}, inplace=True )

In [31]:
x_tests = pd.read_csv('/kaggle/input/titanic/test.csv')

ffff = pd.concat([x_tests['PassengerId'],predictss],axis=1)

In [32]:
final = ffff[['PassengerId','Survived']]
final

In [33]:
from IPython.display import HTML
import pandas as pd
import numpy as np


final.to_csv('submission.csv', index=False)
final.shape

In [34]:
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')

In [35]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train_prepared, y_train)

In [36]:
decision_tree.score(x_train_prepared, y_train)


In [37]:
predicts = decision_tree.predict(x_test_prepared)
predictss = pd.DataFrame(predicts)
predictss.rename( columns={0 :'Survived'}, inplace=True )
x_tests = pd.read_csv('/kaggle/input/titanic/test.csv')

ffff = pd.concat([x_tests['PassengerId'],predictss],axis=1)
final = ffff[['PassengerId','Survived']]
final.to_csv('submission.csv', index=False)
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename='submission.csv')


In [38]:
# Searching for better parameters for logistic regression
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'C':np.logspace(-3,3,7), 'penalty':['l1','l2']}
]
 
log_reg = LogisticRegression()
grid_search = GridSearchCV(log_reg, param_grid, cv=10)
grid_search.fit(x_train_prepared, y_train)
best_search_rf = grid_search.best_estimator_
print(best_search_rf)
grid_search.best_params_

In [39]:
Logistic = LogisticRegression(C= 0.1, penalty= 'l2')
Logistic.fit(x_train_prepared, y_train)

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
model_2 = GradientBoostingClassifier() 
param = {
        'n_estimators': np.arange(400, 1000, 50)
}
grid = GridSearchCV(model_2, param_grid = param, cv=5)

grid.fit(x_train_prepared, y_train)


In [46]:
grid.best_score_

In [47]:
grid.best_estimator_

In [48]:
predicts = grid.predict(x_test_prepared)
predictss = pd.DataFrame(predicts)
predictss.rename( columns={0 :'Survived'}, inplace=True )
x_tests = pd.read_csv('/kaggle/input/titanic/test.csv')

ffff = pd.concat([x_tests['PassengerId'],predictss],axis=1)
final = ffff[['PassengerId','Survived']]
final.to_csv('submission.csv', index=False)
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename='submission.csv')
